# Thermal Radiative Model Interface

This notebook will produce relevant figures from the accompanying publication. The first section uses a data file of measured secondary eclipses. The remaining sections are specific to planets with sampled light curves. It imports existing MCMC results for likelihood optimization of the thermal model parameters.

***
##### Import shared packages.

In [ ]:
import datetime
from glob import glob
from importlib import import_module

import astropy.constants as C
import astropy.units as U
from astropy.analytic_functions import blackbody_lambda
from astropy.stats import bootstrap
import numpy as N
from scipy.stats import linregress
from scipy.stats import lognorm

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import ticker
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
from colorcet import cm

##### Import instrumental response data.

In [ ]:
from data.bandpass.spitzer_IRAC import spitzer_IRAC as spitzer_IRAC
from data.bandpass.JHKs.TwoMASS import TwoMASS as TwoMASS
from data.bandpass.kepler import kepler as kepler
bandpasses = {**kepler.bandpass, **TwoMASS.bandpass, **spitzer_IRAC.bandpass}

***
## Secondary Eclipse Plots

##### Import data on measured eclipse depths in a variety of photometric bands, along with some system properties. We want to calculate the ratio of the measured depths to what the depth should be if the planet were radiating uniformly at its theoretical equilibrium temperature.

In [ ]:
eclipse_file = N.genfromtxt('files/sec_eclipses.csv', delimiter=',', names=True, usecols=(range(1,86)), comments='#')
eclipse_names = N.genfromtxt('files/sec_eclipses.csv', delimiter=',', usecols=0, dtype=str, comments='#')[1:]

In [ ]:
e = eclipse_file['ECC']
w = eclipse_file['OM']
Teff = eclipse_file['TEFF'] * U.K
Teq = eclipse_file['TEFF'] * N.sqrt(0.5/eclipse_file['AR'] * (1-e*N.sin(w))/(1-e**2)) * U.K
Rs = eclipse_file['RS'] * C.R_sun
Rp = eclipse_file['RP'] * C.R_jup

In [ ]:
dTeff_upper = eclipse_file['TEFFUPPER']/eclipse_file['TEFF']
dTeff_lower = eclipse_file['TEFFLOWER']/eclipse_file['TEFF']

dTeq_upper = N.sqrt(dTeff_upper**2 + (0.5*eclipse_file['ARUPPER']/eclipse_file['AR'])**2)
dTeq_lower = N.sqrt(dTeff_lower**2 + (0.5*eclipse_file['ARLOWER']/eclipse_file['AR'])**2)

dRs_upper = eclipse_file['RSUPPER']/eclipse_file['RS']
dRs_lower = eclipse_file['RSLOWER']/eclipse_file['RS']
dRp_upper = eclipse_file['RPUPPER']/eclipse_file['RP']
dRp_lower = eclipse_file['RPLOWER']/eclipse_file['RP']

In [ ]:
flux_ratio = [{}, {}]
dflux_ratio_upper = [{}, {}]
dflux_ratio_lower = [{}, {}]

for band in bandpasses:
    dF_obs_upper = eclipse_file['{0}UPPER'.format(band)]/eclipse_file[band]
    dF_obs_lower = eclipse_file['{0}LOWER'.format(band)]/eclipse_file[band]
    
    for i, n in enumerate([N.sqrt(2),1]):
        waves = bandpasses[band]['wavelength'] * U.um
        spectrum = bandpasses[band]['weighted_spectrum'] * U.um
        
        B_eq = [blackbody_lambda(in_x = waves, temperature = T*n)*U.sr for T in Teq]
        I_eq = N.array([N.dot(B, spectrum) for B in B_eq])*B_eq[0].unit*spectrum.unit
        dI_eq_upper = N.array([(0.5/(C.c*C.k_B*T) * dT * N.sqrt(N.sum((waves**5 * N.exp(C.h*C.c/(waves*C.k_B*T*n)) * B**2)**2)) / I).decompose() for B, I, T, dT in zip(B_eq, I_eq, Teq, dTeq_upper)])
        dI_eq_lower = N.array([(0.5/(C.c*C.k_B*T) * dT * N.sqrt(N.sum((waves**5 * N.exp(C.h*C.c/(waves*C.k_B*T*n)) * B**2)**2)) / I).decompose() for B, I, T, dT in zip(B_eq, I_eq, Teq, dTeq_lower)])
                       
        B_star = [blackbody_lambda(in_x = waves, temperature = T*n)*U.sr for T in Teff]
        I_star = N.array([N.dot(B, spectrum) for B in B_star])*B_star[0].unit*spectrum.unit
        dI_star_upper = N.array([(0.5/(C.c*C.k_B*T) * dT * N.sqrt(N.sum((waves**5 * N.exp(C.h*C.c/(waves*C.k_B*T*n)) * B**2)**2)) / I).decompose() for B, I, T, dT in zip(B_star, I_star, Teff, dTeff_upper)])
        dI_star_lower = N.array([(0.5/(C.c*C.k_B*T) * dT * N.sqrt(N.sum((waves**5 * N.exp(C.h*C.c/(waves*C.k_B*T*n)) * B**2)**2)) / I).decompose() for B, I, T, dT in zip(B_star, I_star, Teff, dTeff_lower)])
                       
        F_th = I_eq / I_star * (Rp/Rs)**2
        
        flux_ratio[i][band] = eclipse_file[band] / F_th
        dflux_ratio_upper[i][band] = N.sqrt(dF_obs_upper**2 + dI_eq_upper**2 + dI_star_upper**2 + 4*(dRs_upper**2 + dRp_upper**2))
        dflux_ratio_lower[i][band] = N.sqrt(dF_obs_lower**2 + dI_eq_lower**2 + dI_star_lower**2 + 4*(dRs_lower**2 + dRp_lower**2))

In [ ]:
T_B = {}
for band in ['3p6', '4p5', '5p8', '8p0']:
    l = float(band.replace('p','.')) * U.um
    l_arg = C.h * C.c / (l * C.k_B)
    flux_ratio[1][band] = ((l_arg / N.log((N.exp(l_arg/Teq)-1)/flux_ratio[1][band] + 1)).to(U.K) / Teq).value

##### Plot a scatterplot of the ratio of observed eclipse depths to the thermal depths, as a function of equilibrium temperature.

In [ ]:
plot_colors = ['#4575b4', '#74add1', '#abd9e9', '#e0f3f8', '#fee090', '#fdae61', '#f46d43', '#d73027']
plot_labels = [r'Kepler', r'$J$', r'$H$', r'$K_s$', r'3.6 $\mu$m', r'4.5 $\mu$m', r'5.8 $\mu$m', r'8.0 $\mu$m']

In [ ]:
#Epsilon parameter is 0 for zero redistribution, 1 for perfect redistribution.
ep = 1

fig, ax = plt.subplots(figsize=(6,6))

for i, band in enumerate(bandpasses):
    ax.scatter(Teq, flux_ratio[ep][band], c=plot_colors[i], marker='.')
    ax.errorbar(Teq/U.K, flux_ratio[ep][band], xerr=Teq/U.K*N.array([dTeq_lower, dTeq_upper]), yerr=N.array([dflux_ratio_lower[ep][band],dflux_ratio_upper[ep][band]])*flux_ratio[ep][band], c=plot_colors[i], label=plot_labels[i], fmt='.', elinewidth=0.5)
    
ax.axhline(1, c='k', linewidth=0.5, linestyle='dotted', alpha=0.5)

kepler_low = N.nanargmin(flux_ratio[ep]['KP'])
kepler_high = N.nanargmax(flux_ratio[ep]['KP'])
IRAC8p0_low = N.nanargmin(flux_ratio[ep]['8p0'])

ax.annotate('', ha = 'center', va = 'center', color='k',
            xy=(float(Teq[kepler_high]/U.K)+200, N.nanmax(flux_ratio[ep]['KP'])-100), xycoords='data',
            xytext=(float(Teq[kepler_low]/U.K), N.nanmin(flux_ratio[ep]['KP'])), textcoords='data', fontsize=10,
            arrowprops=dict(arrowstyle="fancy",
                            color="k", alpha=0.1,
                            mutation_scale=50,
                            shrinkA=20,
                            shrinkB=0,
                            connectionstyle="angle3,angleA=180,angleB=-90",
                            ),
            )

ax.annotate('', ha = 'center', va = 'center', color='k',
            xy=(float(Teq[IRAC8p0_low]/U.K), N.nanmin(flux_ratio[ep]['8p0'])), xycoords='data',
            xytext=(float(N.nanmax(Teq)/U.K), flux_ratio[ep]['4p5'][N.nanargmax(Teq)]), textcoords='data', fontsize=10,
            arrowprops=dict(arrowstyle="fancy",
                            color="k", alpha=0.1,
                            mutation_scale=50,
                            shrinkA=20,
                            shrinkB=10,
                            connectionstyle="angle3,angleA=180,angleB=90",
                            ),
            )

ax.text(2375, 0.6, 'Clouds\ndecrease\nemissivity', ha = 'center', va = 'center',
                    color='k', alpha=0.5, fontsize=12)

ax.text(2000, 60, 'Clouds\nincrease\nreflectivity', ha = 'center', va = 'center',
                    color='k', alpha=0.5, fontsize=12)

plt.xlabel(r'T$_\mathrm{eq}$ (K)', fontsize=18)
plt.ylabel(r'F$_\mathrm{obs}$/F$_\mathrm{eq}$', fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=12)
ax.set_yscale('log')
ax.set_ylim([0.35, 500])
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%d"))

plt.legend(loc='upper right', fontsize=10, ncol=1)

plt.savefig('eclipse_scatterplot.pdf', bbox_inches='tight')

##### Plot a histogram of the eclipse ratios, including a best-fit log-normal distribution.

In [ ]:
ep = 1

hist_arrays = {}
for band in bandpasses:
    #Plotting cut for the histogram above a ratio of 3.
    hist_arrays[band] = (flux_ratio[ep][band])[flux_ratio[ep][band]< 3.]
bins = N.histogram(N.hstack(hist_arrays.values()), bins=25)[1]
lognorm_fit = lognorm.fit(N.hstack(hist_arrays.values()), loc=N.log(1.2))
#lognorm_fit = lognorm.fit(N.hstack([hist_arrays['3p6'], hist_arrays['4p5'], hist_arrays['5p8'], hist_arrays['8p0']]), loc=N.log(1.2))

plt.hist(hist_arrays.values(),
         bins=bins, color=plot_colors,
         label=plot_labels,
         stacked=True, normed=True)

x = N.linspace(0.5, 3, 100)
plt.plot(x, lognorm.pdf(x, *lognorm_fit), c='#444444', linewidth=3, linestyle='dotted', alpha=0.5)

plt.text(0.95, 0.5, '$x_0=%.3f$\n$\mu=%.3f$\n$\sigma=%.3f$'%((lognorm_fit[1], N.log(lognorm_fit[2]), lognorm_fit[0])),
         transform=plt.gca().transAxes, fontsize=12, horizontalalignment='right', verticalalignment='center')
plt.legend(loc='upper right', fontsize=10, ncol=2)
plt.xlabel(r'F$_\mathrm{obs}$/F$_\mathrm{eq}$', fontsize=16)
plt.ylabel(r'Normalized Frequency', fontsize=16)

plt.savefig('eclipse_histogram.pdf', bbox_inches='tight')

##### We can construct the equivalent of a color-magnitude diagram for eclipse depths, where magnitudes are logarithms of eclipse ratios.

In [ ]:
def CMD(bands):
    mask = [N.argwhere(N.isfinite(ratio[bands[0]]) & N.isfinite(ratio[bands[1]])).squeeze() for ratio in flux_ratio]
    x = [N.log(ratio[bands[1]][idx]) for idx, ratio in zip(mask, flux_ratio)]
    y = [(N.log(ratio[bands[0]])-N.log(ratio[bands[1]]))[idx] for idx, ratio in zip(mask, flux_ratio)]
    teq = [Teq[idx] for idx, ratio in zip(mask, flux_ratio)]
    return {'idx': mask, 'mag': x, 'color': y, 'Teq': teq}

In [ ]:
labeled_planets = ['HD 209458 b', 'TrES-3 b', 'HD 189733 b', 'HAT-P-2 b', 'XO-3 b']
label_indices = [int(N.argwhere(eclipse_names==[name])) for name in labeled_planets]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(13, 4), sharex=True, sharey=True)
CMDs = [CMD(['3p6', '4p5']), CMD(['4p5', '5p8']), CMD(['5p8', '8p0'])]
plot_bands = [['3.6', '4.5'], ['4.5', '5.8'], ['5.8', '8.0']]

quantity = 'Teq'

Teq_min = N.nanmin([N.nanmin([cmd[quantity][0], cmd[quantity][1]]) for cmd in CMDs])
Teq_max = N.nanmin([N.nanmax([cmd[quantity][0], cmd[quantity][1]]) for cmd in CMDs])

for i, (ax, cmag) in enumerate(zip(axes, CMDs)):
    scatter_ep0 = ax.scatter(cmag['mag'][0], cmag['color'][0], marker='X', c='#888888', vmin=Teq_min, vmax=Teq_max, label='$\epsilon=0$')
    scatter_ep1 = ax.scatter(cmag['mag'][1], cmag['color'][1], c=cmag[quantity][1], vmin=Teq_min, vmax=Teq_max, cmap=cm['isoluminant_cgo_70_c39'], label='$\epsilon=1$')
    for x0, y0, x1, y1, color in zip(cmag['mag'][0], cmag['color'][0], cmag['mag'][1], cmag['color'][1], cmag['Teq'][1]):
        ax.plot([x0,x1],[y0,y1], c='k', linewidth=0.5, alpha=0.3, zorder=0)

    ax.set_xlabel(r'$\log \left(F_\mathrm{{obs}}/F_\mathrm{{eq}}\right)$, {0}$\mu$m'.format(plot_bands[i][1]),fontsize=16)
    ax.set_ylabel(r'$\log \left(F_\mathrm{{o}}/F_\mathrm{{e}}\right)$, {0}$\mu$m - $\log \left(F_\mathrm{{o}}/F_\mathrm{{e}}\right)$, {1}$\mu$m'.format(*plot_bands[i]), fontsize=13)
    
    ax.axvline(0, c='k', linewidth=0.5, linestyle='dotted', alpha=0.5)
    x = N.linspace(*ax.get_xlim())
    ax.plot(x, -x, c='k', linewidth=0.5, linestyle='dotted', alpha=0.5)
    
    ax.set_xlim([-0.5,1.25])
    ax.set_ylim([-0.75,1.25])
    ax.add_artist(FancyArrowPatch((-0.01,0.5*(ax.get_ylim()[1])), (0.15,0.5*(ax.get_ylim()[1])), arrowstyle='simple', mutation_scale=10, edgecolor=None, facecolor='k', alpha=0.5, zorder=1))
    ax.add_artist(FancyArrowPatch((-0.5*(ax.get_ylim()[0])-0.01,0.5*(ax.get_ylim()[0])), (-0.5*(ax.get_ylim()[0])-0.01+0.15/N.sqrt(2),0.5*(ax.get_ylim()[0])+0.15/N.sqrt(2)), arrowstyle='simple', mutation_scale=10, edgecolor=None, facecolor='k', alpha=0.5, zorder=1))
    
    ax.set_aspect('equal')
    
    for idx, name in zip(label_indices, labeled_planets):
        if idx in cmag['idx'][0] and idx in cmag['idx'][1]:
            x0_ = cmag['mag'][0][cmag['idx'][0] == idx]
            y0_ = cmag['color'][0][cmag['idx'][0] == idx]
            x1_ = cmag['mag'][1][cmag['idx'][1] == idx]
            y1_ = cmag['color'][1][cmag['idx'][1] == idx]
            
            text_angle = float(N.arctan((y1_-y0_)/(x1_-x0_))/U.deg)
            line_length = N.sqrt((x1_-x0_)**2+(y1_-y0_)**2)
            
            ax.annotate(name, ha = 'center', va = 'center', color='k', rotation=text_angle,
            xy=(0.5*(x0_+x1_),0.5*(y0_+y1_)), xycoords='data',
            xytext=(0.5*(x0_+x1_)-0.025*(y1_-y0_)/line_length,0.5*(y0_+y1_)+0.025*(x1_-x0_)/line_length), textcoords='data', fontsize=6,
            )
    
fig.subplots_adjust(right=0.88)
cbar_ax = fig.add_axes([0.89, 0.15, 0.025, 0.55])
cb = fig.colorbar(scatter_ep1, cax=cbar_ax)
cb.set_label(r'$T_\mathrm{eq}$ (K)', fontsize=16)

legend_ax = fig.add_axes([0.8835, 0.7, 0.025, 0.2], frameon=False)
legend_ax.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
legend_ax.grid(False)
p,l = axes[-1].get_legend_handles_labels()
legend = legend_ax.legend(p, l, loc='center left', fontsize=11)
legend.legendHandles[0].set_color('k')
legend.legendHandles[1].set_color('k')

plt.savefig('eclipse_CMD_{0}.pdf'.format(quantity), bbox_inches='tight')

***
## Planet-specific Modules

##### The planet class contains all the methods used for the geometry of the planet-star system.

In [ ]:
from planet_class import Planet

##### Import the system properties and data of the planets (contained in Python dictionaries in a separate file), into a dictionary of planet classes. For some plots we just want to pick out a specific planet, which we specify here.

In [ ]:
paths = [s.split('/')[-1] for s in glob('data/planet/*') if '__' not in s]
planets = {}
for path in paths:
    directory = import_module('data.planet.{0}.{0}'.format(path))
    if directory.data != None:
        planets[path] = Planet(directory)

specific_name = 'HATP7b'

##### Import the module for the thermal model.

In [ ]:
from thermal import blackbody as thermal_model

##### Import the routine to convert surface temperatures to observed planet-star flux ratios in a given band.

In [ ]:
from data.bandpass.response import light_curve

##### Import the likelihood calculation routine.

In [ ]:
from stats.gaussian import log_likelihood
from stats.metropolis import MCMC

##### Import any visualization routines needed.

In [ ]:
from visualization import multi_animation
from visualization.style import colors
from visualization.lightcurve_plot import LightCurvePlot
from visualization.orbit_plot import OrbitPlot
from visualization.surface_plot import SurfacePlot

***
## Best-Fit Model Light Curves from Existing MCMC Results

##### A wrapper function using the thermal model and light curve routines to start with a set of specified parameter values and return a light curve. This function will be passed to the statistical likelihood function.

In [ ]:
def generate_model(planet, parameters, spectral_array):
    temperature_map = thermal_model.temperatures(planet=planet, parameters=parameters)
    model = light_curve(planet=planet,
                        temp_map=temperature_map,
                        spectral_array=spectral_array,
                        parameters=parameters,
                        use_tidal_distortion=False)
    return {'temp': temperature_map, 'model': model}

##### If there are saved MCMC outputs we want to use, import them here. Usually we have one run which converges (hopefully) to the optimum point in parameter space, as well as a run that starts at that optimum and freely explores the region around the optimum to get an estimate on the uncertainty.

##### For the MCMC routine we feed the parameters in as a list. The parameter order is:
1. Rotation period
2. Radiative timescale
3. Minimum temperature
4. Albedo

In [ ]:
mcmc_file = {}
mcmc_uncertainty_file = {}
mcmc_fit = {}
mcmc_uncertainty = {}
sigma_upper = {}
sigma_lower = {}

for name, planet in sorted(planets.items()):
    try:
        mcmc_file[name] = N.load('files/{0}_mcmc.npy'.format(planet.name.replace(" ", "")), encoding='latin1', fix_imports=True)[()]
        mcmc_uncertainty_file[name] = N.load('files/{0}_mcmc_uncertainties.npy'.format(planet.name.replace(" ", "")), encoding='latin1', fix_imports=True)[()]
    except:
        continue
    
    print(name)
    
    mcmc_fit[name] = {}
    mcmc_uncertainty[name] = {}
    sigma_upper[name] = {}
    sigma_lower[name] = {}
    
    for band in planet.data:
        mcmc_fit[name][band] = mcmc_file[name][band]['pos'][-1].squeeze() * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        print('Band: {0} um'.format(band.replace('p', '.')))
        print('Best-fit parameters:\n Rotation period = {0:.2f} ({4:.2f} PSR)\n Radiative timescale @ {5:.0f} = {1:.1f}\n Minimum temperature = {2:.0f}\n Albedo = {3:.2f}\n'.format(*mcmc_fit[name][band], mcmc_fit[name][band][0]/planet.pseudosynchronous_period(), planet.orbital_equilibrium_temperature().decompose()))
        mcmc_uncertainty[name][band] = mcmc_uncertainty_file[name][band]['pos'].squeeze()
    
        #We take the explored parameter space from the uncertainty run and generate 1-sigma bounds around the best-fit values.
        param_upper = []
        param_lower = []
        for i, fit in enumerate(mcmc_uncertainty[name][band][0]):
            upper_entries = (mcmc_uncertainty[name][band].T[i])[mcmc_uncertainty[name][band].T[i] > fit]
            if upper_entries.size == 0:
                param_upper.append(N.array([N.nan]))
            else:
                param_upper.append(upper_entries)
        for i, fit in enumerate(mcmc_uncertainty[name][band][0]):
            lower_entries = (mcmc_uncertainty[name][band].T[i])[mcmc_uncertainty[name][band].T[i] < fit]
            if lower_entries.size == 0:
                param_lower.append(N.array([N.nan]))
            else:
                param_lower.append(lower_entries)
            
        sigma_upper[name][band] = N.array([N.nanpercentile(p, 68) - mcmc_uncertainty[name][band][0][i] for i, p in enumerate(param_upper)]) * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        sigma_lower[name][band] = N.array([N.nanpercentile(p, 100-68) - mcmc_uncertainty[name][band][0][i] for i, p in enumerate(param_lower)]) * [planet.pseudosynchronous_period(), U.hr, U.K, U.dimensionless_unscaled]
        print('1-sigma bounds:\n Rotation period = +{0:.2f} ({4:.2f} PSR), {5:.2f} ({9:.2f} PSR)\n Radiative timescale @ {10:.0f} = +{1:.1f}, {6:.1f}\n Minimum temperature = +{2:.0f}, {7:.0f}\n Albedo = +{3:.2f}, {8:.2f}\n'.format(*sigma_upper[name][band], sigma_upper[name][band][0]/planet.pseudosynchronous_period(), *sigma_lower[name][band], sigma_lower[name][band][0]/planet.pseudosynchronous_period(), planet.orbital_equilibrium_temperature().decompose()))

##### Here we use the entire dictionary of planets to create a text file with the formatted table information for the publication.

In [ ]:
table_name = 'parameter_results.txt'
    
with open(table_name, 'w+') as table_file:
    for name, planet in sorted(planets.items()):
        if len(planet.data) > 1:
            row = '\multirow{{{0}}}{{*}}{{{1}}}'.format(len(planet.data), planet.name)
        else:
            row = planet.name
        #We test whether the planet has associated MCMC files. Otherwise, just don't include the results in the table.
        try:
            for band in planet.data:
                row_band = ' & ' + band.replace('p', '.') + ' & '
                row_band += '${0:.2f}^{{+{1:.2f}}}_{{{2:.2f}}}$ & '.format(mcmc_fit[name][band][0]/planet.pseudosynchronous_period(),
                                                                    sigma_upper[name][band][0]/planet.pseudosynchronous_period(),
                                                                    sigma_lower[name][band][0]/planet.pseudosynchronous_period())
                if round(mcmc_fit[name][band][3].value, 2) > 0.00:
                    row_band += '${0:.2f}^{{+{4:.2f}}}_{{{8:.2f}}}$ & ${1:.1f}^{{+{5:.1f}}}_{{{9:.1f}}}$ & ${2:.0f}^{{+{6:.0f}}}_{{{10:.0f}}}$ & ${3:.2f}^{{+{7:.2f}}}_{{{11:.2f}}}$'\
                                .format(*[x.value for x in mcmc_fit[name][band]], *[x.value for x in sigma_upper[name][band]], *[x.value for x in sigma_lower[name][band]])
                else:
                    row_band += '${0:.2f}^{{+{3:.2f}}}_{{{7:.2f}}}$ & ${1:.1f}^{{+{4:.1f}}}_{{{8:.1f}}}$ & ${2:.0f}^{{+{5:.0f}}}_{{{9:.0f}}}$ & $<{6:.2f}$'\
                                .format(*[x.value for x in mcmc_fit[name][band]][:-1], *[x.value for x in sigma_upper[name][band]], *[x.value for x in sigma_lower[name][band]][:-1])
                row_band += r' \\'
                row += row_band
        except:
            continue
        row = row.replace('nan', 'N/A')
        row += '\n' + r'\hline' + '\n'
        table_file.write(row)

##### For a specific planet: quick plots showing the trajectory of the MCMC walker(s) in each of the parameter dimensions, as well as the evolution of the likelihood.

In [ ]:
fig, axes = plt.subplots(5, 2, sharex=True, figsize=(12, 12))
name = specific_name
for i in range(4):
    for band in planets[name].data:
        axes[i][0].plot((mcmc_file[name][band]['pos'].T)[i][0], color=colors.color_datlab[band])
        axes[4][0].plot(mcmc_file[name][band]['logl'], color=colors.color_datlab[band])
        axes[0][0].set_title(r'Convergence Run', fontsize=20)
        
        axes[0][0].set_ylabel(r'$P_{\mathrm{rot}}/P_{\mathrm{PSR}}$', fontsize=16)
        axes[1][0].set_ylabel(r'$\tau_{\mathrm{rad}}$ (hr)', fontsize=16)
        axes[2][0].set_ylabel(r'$T_{0}$ (K)', fontsize=16)
        axes[3][0].set_ylabel(r'Albedo', fontsize=16)
        axes[4][0].set_ylabel(r'Log Likelihood', fontsize=14)
        
        axes[i][1].plot((mcmc_uncertainty_file[name][band]['pos'].T)[i][0], color=colors.color_datlab[band])
        axes[4][1].plot(mcmc_uncertainty_file[name][band]['logl'], color=colors.color_datlab[band])
        axes[0][1].set_title(r'Uncertainty Run', fontsize=20)

##### Pull the most favorable parameter values and make a grid of that single set to plot the flux curve(s). Then generate models for the most favorable parameters in each band.

In [ ]:
best_parameters = {}
best_rotation = {}
best_paramgrid = {}
best_model = {}

name = specific_name
for band in planets[name].data:
    best_parameters[band] = mcmc_fit[name][band]
    best_rotation[band] = best_parameters[band][0]
    best_paramgrid[band] = N.meshgrid(*best_parameters[band])
    
    #A time resolution of 200 is used for plotting for all planets except HD 80606 b, which uses 500.
    planets[name].set_resolution(longitude_resolution = 72,
                                   latitude_resolution = 36,
                                   time_resolution = 200,
                                   num_orbits = 5)
    
    if name == 'HD80606b':
        planets[name].time_resolution = 500
        planets[name].times = N.array(N.concatenate([N.r_[N.linspace((i-0.5)*(planets[name].P/U.d), -1.25+i*(planets[name].P/U.d), num=125, endpoint=False), N.linspace(-1.25+i*(planets[name].P/U.d), 1.25+i*(planets[name].P/U.d), num=250, endpoint=False), N.linspace(1.25+i*(planets[name].P/U.d), (i+0.5)*(planets[name].P/U.d), num=125, endpoint=False)] for i in range(5)])) * U.d
    
    best_model[band] = generate_model(planets[name], best_paramgrid[band], spitzer_IRAC.bandpass[band])

##### For our transit depth analysis, we can optionally modify the data by translating the in-transit data up by the geometric transit depth. Then, use delta_data instead of planet.data.

In [ ]:
delta_data = {}
for band in planets[name].data:
    delta_data[band] = {'t':planets[name].data[band]['t'],
                        'flux': None,
                        'occultation':planets[name].data[band]['occultation']}

transit_dip = (planets[name].rp/planets[name].R)**2
print('Geometric transit depth: {0}'.format(transit_dip))

for band in delta_data:
    delta_data[band]['flux'] = N.where(planets[name].data[band]['occultation']==b't',
                                       planets[name].data[band]['flux'] + transit_dip,
                                       planets[name].data[band]['flux'])

##### Initialize the light curve with our best-fit models in each model, then draw with arbitrary smoothing for the shaded region representing rough 1- and 2-sigma boundaries on the model light curves.

In [ ]:
uncertainty_models = N.load('files/uncertainty_lightcurves.npy', encoding='latin1', fix_imports=True)[()]

In [ ]:
if name == 'HD80606b':
    sigma_curves = {'resolution': 500,
                    'times': N.array(N.concatenate([N.r_[N.linspace((i-0.5)*(planets[name].P/U.d), -1.25+i*(planets[name].P/U.d), num=125, endpoint=False), N.linspace(-1.25+i*(planets[name].P/U.d), 1.25+i*(planets[name].P/U.d), num=250, endpoint=False), N.linspace(1.25+i*(planets[name].P/U.d), (i+0.5)*(planets[name].P/U.d), num=125, endpoint=False)] for i in range(5)])) * U.d}
else:
    sigma_curves = {'resolution': 200,
                    'times': planets[name].times}
    
for band in planets[name].data:
    models = N.array(uncertainty_models[name][band]['model'])
    counts = uncertainty_models[name][band]['count']
    flux_step = N.squeeze(N.repeat(models, counts, axis=0))
    sigma_curves[band] = {'lower': [], 'upper': []}
    for sigma in [68, 95]:    
        lower_flux = N.nanpercentile(flux_step, 50-sigma/2, axis=0)
        sigma_curves[band]['lower'].append(lower_flux)
        upper_flux = N.nanpercentile(flux_step, 50+sigma/2, axis=0)
        sigma_curves[band]['upper'].append(upper_flux)

In [ ]:
lightcurve_plot = LightCurvePlot(planet=planets[name],
                                 #data=planets[name].data,
                                 data=delta_data,
                                 model=best_model,
                                 sigma_bounds = sigma_curves,
                                 parameters=best_parameters)

In [ ]:
fig, ax = plt.subplots()
lightcurve_plot.draw(axis=ax,
                     phase_overlap=0.25,
                     filetypes=['pdf'],
                     combo=False,
                     save=True)

***
## Orbit and Surface Plots

##### Draw a top-down orbit plot with equally time-spaced points.

In [ ]:
name = specific_name
#A time resolution of 32 is used for all planets except HD 80606 b, which uses 500.
if name == 'HD80606b':
    orbit_resolution = 500
else:
    orbit_resolution = 32

orbit_plot = OrbitPlot(planets[name], time_resolution = orbit_resolution)

In [ ]:
fig, ax = plt.subplots()
orbit_plot.draw_static(axis=ax, filetypes=['pdf'], save=True, use_data=False)

##### Draw a projection of the planet surface with a quantity (e.g. temperature) plotted as a color map. Since we're going to animate this with the orbit plot, which has a time resolution different from the generated model time resolution, we need to select out every nth timestep, n given by the ratio.

In [ ]:
selected_band = '4p5'

if name == 'HD80606b':
    surface_resolution = 500
else:
    surface_resolution = 32

surface_plot = SurfacePlot(planets[name],
                           rotation_period = best_rotation[selected_band],
                           map_array = (best_model[selected_band]['temp'].reshape(planets[name].time_resolution*(planets[name].num_orbits),(planets[name].latitude_resolution)+1,(planets[name].longitude_resolution)+1)),
                           time_resolution = surface_resolution,
                           start_orbit = 3)

##### The multi_animation routine takes multiple visualizations and animates them together over a time series.

In [ ]:
orbit_plot.set_rotation_period(planets[name].P)
orbit_plot.set_num_orbits(3)
animation = multi_animation.draw(planets[name],
                                 plot_objects = [orbit_plot, surface_plot],
                                 num_orbits = 3)

***
## Parameter Plots

##### One interesting plot is to look at a map of likelihoods in a 2D slice of parameter space. Here, we make a plot of rotation period versus radiative timescale, using an existing grid search with the other 2 parameters (minimum temperature, albedo) at values fixed to their best fits from MCMC. This shows us how degenerate the selection of the 2 timescale parameters is.

In [ ]:
name = specific_name
degeneracy_file = N.load('files/{0}_grid.npy'.format(name), encoding='latin1', fix_imports=True)[()]

In [ ]:
pars = degeneracy_file['par'][:]
pars[0] = (pars[0]/planets[name].P).decompose()

degeneracy_grid = degeneracy_file['logl']['4p5'].squeeze()
best_index = N.unravel_index(N.argmin(degeneracy_grid), (len(pars[0]), len(pars[1])))
logl_min = N.nanmin(degeneracy_grid)

x, y = N.mgrid[pars[0][0]:pars[0][-1]:len(pars[0])*1j, pars[1][0]:pars[1][-1]:len(pars[1])*1j]
plt.pcolor(x, y, degeneracy_grid/logl_min, cmap='cet_fire_r', vmin=1, vmax=1.5)
cb = plt.colorbar()
cb.set_label('Scaled Log Likelihood', fontsize=16)
plt.xlabel(r'$P_{\mathrm{rot}}/P_{\mathrm{orb}}$', fontsize=16)
plt.ylabel(r'$\tau_{\mathrm{rad}}$ (hr)', fontsize=16)
plt.savefig('{0}_degeneracy.pdf'.format(name), bbox_inches='tight', transparent=True)

##### We also plot scatterplots of best-fit parameters versus various properties of the system. Here we show two plots: one of the peak instellation versus the best-fit rotation period, and the time-averaged instellation versus the radiative timescale expressed at the periastron equilibrium temperature.

In [ ]:
instellation = {'max': {}, 'maxerr': {},
                'mean': {}, 'meanerr': {}}

for name in planets:
    n = planets[name].name
    entry = eclipse_file[eclipse_names==n]
    
    TEFF = entry['TEFF'][0] / 5776
    dTEFF_l = entry['TEFFLOWER'][0] / entry['TEFF'][0]
    dTEFF_u = entry['TEFFUPPER'][0] / entry['TEFF'][0]
    
    a_jup = 5.20336301 * U.AU
    aR = entry['AR'][0] / (a_jup/C.R_sun).decompose()
    daR_l = entry['ARLOWER'][0] / entry['AR'][0]
    daR_u = entry['ARUPPER'][0] / entry['AR'][0]
    
    e = entry['ECC'][0]
    instellation['mean'][name] = TEFF**4 / aR**2 / N.sqrt(1-e**2)
    instellation['max'][name] = TEFF**4 / aR**2 / (1-e)**2
    instellation['meanerr'][name] = [N.sqrt((4*(TEFF**4)*dTEFF_l)**2 + (2*(aR**-2)*daR_l)**2 + (e/(1-e**2)**1.5)**2),
                                 N.sqrt((4*(TEFF**4)*dTEFF_u)**2 + (2*(aR**-2)*daR_u)**2 + (e/(1-e**2)**1.5)**2)]
    instellation['maxerr'][name] = [N.sqrt((4*(TEFF**4)*dTEFF_l)**2 + (2*(aR**-2)*daR_l)**2 + (2/(1-e)**3)**2),
                                 N.sqrt((4*(TEFF**4)*dTEFF_u)**2 + (2*(aR**-2)*daR_u)**2 + (2/(1-e)**3)**2)]

In [ ]:
quantity_bands = {}
planet_links = {}
for band in ['3p6', '4p5', '8p0']:
    quantity_bands[band] = {'x': [], 'y': [], 'xerr': [], 'yerr': []}

quantity = 'mean'

for (name, par), (name, dn), (name, up) in zip(mcmc_fit.items(), sigma_lower.items(), sigma_upper.items()):
    planet_links[name] = {'x': [], 'y': []}
    for band in par:
        x = instellation[quantity][name]
        xerr = instellation[quantity+'err'][name]
        
        quantity_bands[band]['x'].append(x)
        planet_links[name]['x'].append(x)
        
        quantity_bands[band]['xerr'].append(xerr)
        
        if quantity == 'mean':
            y = par[band][1]/U.hr * (1-planets[name].e)**1.5
            yerr = [-dn[band][1]/U.hr*(1-planets[name].e)**1.5, up[band][0]/U.hr*(1-planets[name].e)**1.5]
            
        else:
            y = par[band][0].to(U.d)/planets[name].pseudosynchronous_period()
            yerr = [-dn[band][0].to(U.d)/planets[name].pseudosynchronous_period(), up[band][0].to(U.d)/planets[name].pseudosynchronous_period()]
        
        quantity_bands[band]['y'].append(y)
        planet_links[name]['y'].append(y)
        
        quantity_bands[band]['yerr'].append(yerr)

##### To quantify any potential correlation, we resample the data many times with replacement, varying the position of the drawn points within the propagated error ranges.

In [ ]:
num_bootstrap = 10**4

x_cum = N.concatenate([quantity_bands[band]['x'] for band in quantity_bands])
xerr_cum = N.concatenate([(quantity_bands[band]['x']+(N.array([-1,1])*quantity_bands[band]['xerr']).T).T for band in quantity_bands])
y_cum = N.concatenate([quantity_bands[band]['y'] for band in quantity_bands])
yerr_cum = N.concatenate([(quantity_bands[band]['y']+(N.array([-1,1])*N.nan_to_num(quantity_bands[band]['yerr'])).T).T for band in quantity_bands])

index_sample = bootstrap(N.arange(len(x_cum)), bootnum=num_bootstrap).astype(int)
x_sample = N.log10(N.random.uniform(*(N.einsum('spl->lsp',xerr_cum[index_sample]))))
y_sample = N.log10(N.random.uniform(*(N.einsum('spl->lsp',yerr_cum[index_sample]))))

R_ = N.array([linregress(x_,y_).rvalue**2 for x_, y_ in zip(x_sample, y_sample)])

##### The plots show the results of the regression resampling as insets within the scatterplots.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

for i, band in enumerate(quantity_bands):
    ax.errorbar(quantity_bands[band]['x'], quantity_bands[band]['y'],
                xerr=N.array(quantity_bands[band]['xerr']).T, yerr=N.array(quantity_bands[band]['yerr']).T,
                c=colors.color_modbg[band], fmt='x', capsize=5, elinewidth=2)
    
ax.set_xlabel(r'$F_\star/F_{\odot,J}$', fontsize=20)
ax.set_xscale('log')
if quantity == 'mean':
    ax.set_xlabel(r'$\bar{F_\star}/F_{\odot,J}$', fontsize=20)
    ax.set_ylabel(r'$\tau_{\mathrm{peri}}$ (hr)', fontsize=20)
    ax.set_yscale('log')
    loc=4
else:
    ax.set_xlabel(r'$F_{\star\mathrm{, max}}/F_{\odot,J}$', fontsize=20)
    ax.set_ylabel(r'$P_{\mathrm{orb}}/P_{\mathrm{PSR}}$', fontsize=20)
    loc=2
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

ax_inset = inset_axes(ax, width="30%", height="30%", borderpad=1, loc=loc)
ax_inset.hist(R_, bins=50, label=r'$\bar{{R^2}}={0:.2f}$'.format(N.mean(R_)), color='#412AA0')
ax_inset.set_xlim([0,1])
ax_inset.xaxis.set_ticks(N.linspace(0,1,num=6))
if quantity == 'mean': ax_inset.xaxis.tick_top()
ax_inset.yaxis.set_visible(False)

from mpl_toolkits.axes_grid.anchored_artists import AnchoredText
at = AnchoredText(r'Median $R^2={0:.2f}$'.format(N.median(R_)),
                  prop=dict(size=8), frameon=True,
                  loc=1,
                  )
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax_inset.add_artist(at)

if quantity == 'mean':
    plt.savefig('quants_X_instellationmean_Y_radperi.pdf', bbox_inches='tight')
else:
    plt.savefig('quants_X_instellationpeak_Y_rotper.pdf', bbox_inches='tight')